In [ ]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from dateutil import parser
from datetime import datetime 
import pandas as pd
from sapextractor import database_connection
from sapextractor.utils.dates import timestamp_column_from_dt_tm
from pm4pymdl.objects.ocel.exporter import exporter as ocel_exporter

con = database_connection.oracle.apply()
mandt = "800"

MAKE_ADVANCED_CONNECTIONS = False

## Gather all table data

In [ ]:
# Get all tables

# Change Documents (used for Production Order Updates)
jcds = con.prepare_and_execute_query("JCDS",["OBJNR","STAT","UDATE","UTIME","CHIND"], additional_query_part=" WHERE MANDT = '"+mandt+"'")
# System Status Texts (used to map Production Order STATus to Description Text)
TJ02T = con.prepare_and_execute_query("TJ02T",["ISTAT","TXT30"],additional_query_part=" WHERE SPRAS = 'E'");
# Order Header Data (used for Production Order Information, especially collecting reservation number)
afko = con.prepare_and_execute_query("AFKO", ["AUFNR", "RSNUM"])
afko['AUFNR'] = "OR" + afko['AUFNR'] # Prefix Order ID with 'OR', like it is in JCDS
afko['event_RESERVATION'] = None #TODO: needed?
# Reservatio Requirements (used to connect Production Order with Reservation + required Material)
RESB = con.prepare_and_execute_query("RESB",["RSNUM","RSPOS","MATNR","BANFN","AUFNR","BAUGR","EBELN","BNFPO","EBELP"],additional_query_part="")
RESB['AUFNR'] = "OR" + RESB['AUFNR'] # Prefix Order ID with 'OR', like it is in JCDS
RESB['event_RESERVATION'] = None #TODO: needed? 
RESB['DOCTYPE_RESERVATION'] = None
# General Change Tables (used to extract data for purchase order/requisitions)
cdpos = con.prepare_and_execute_query("CDPOS", ["OBJECTID", "OBJECTCLAS", "CHANGENR", "FNAME","VALUE_NEW", "VALUE_OLD", "TABNAME", "TABKEY", "CHNGIND"], additional_query_part=" WHERE MANDANT = '"+mandt+"'");
cdhdr = con.prepare_and_execute_query("CDHDR", ["OBJECTID","CHANGENR", "UDATE", "UTIME"], additional_query_part=" WHERE UDATE != '00000000' AND MANDANT = '"+mandt+"'");
changes = cdpos.merge(cdhdr,left_on=["OBJECTID","CHANGENR"],right_on=["OBJECTID","CHANGENR"],how="inner")
# Main Tables for Purchase Documents
# Gather all Purchase Orders with prior Purchase Requisition
ekpo = con.prepare_and_execute_query("EKPO",["EBELN","MATNR","MENGE", "MEINS", "BANFN"],additional_query_part=" WHERE MANDT = '"+mandt+"' AND BANFN != ' '");
eban = con.prepare_and_execute_query("EBAN", ["BANFN", "BADAT","MATNR","RSNUM"])
# Goods Receipt for Purchase Orders
gr_for_purord = con.prepare_and_execute_query("EKBE",["EBELN","CPUDT","CPUTM","MATNR"], additional_query_part=" WHERE (NOT CPUDT = '00000000') AND MANDT = '"+mandt+"'")
# Goods Issue for Production Order
gi_for_prodord = con.prepare_and_execute_query("MSEG",["MBLNR","MATNR","MENGE","MEINS","AUFNR","CPUDT_MKPF","CPUTM_MKPF","BWART","RSNUM","RSPOS"], additional_query_part=" WHERE AUFNR > '000000000000' AND MANDT = '"+mandt+"' AND (NOT CPUDT_MKPF = '00000000') AND BWART = '261'")
# Connect Production Order with Reservation + required Material
prodord_reqmat = afko.merge(RESB,how="inner",on=['AUFNR','RSNUM'])
prodord_reqmat['event_RESERVATION'] = None
prodord_reqmat['DOCTYPE_RESERVATION'] = None
if MAKE_ADVANCED_CONNECTIONS:
    # For every Production Order ID (AUFNR), write reservation + req. materials into the event_RESERVATION column of the afko df
    for name, group in prodord_reqmat.groupby(['AUFNR']):
        # If at least one required material is connected and the Order ID is found in the AFKO table
        if len(group['MATNR'].values) >= 1 and len(afko.index[afko['AUFNR'] == name]) >= 1:
            afko_index = afko.index[afko['AUFNR'] == name][0]
            afko.at[afko_index,'event_RESERVATION'] = ["RES" + afko.at[afko_index,'RSNUM'] + '_' + mat for mat in group['MATNR'].values]
    afko['DOCTYPE_RESERVATION'] = afko['event_RESERVATION']

# Connect Production Order Updates with Update Description
prodord_updates = jcds.merge(TJ02T,how='inner',left_on='STAT',right_on='ISTAT').drop_duplicates(subset=['UDATE','UTIME','OBJNR'])
# Augment <<Creation of Production Order>> Events with Reservation + required Material
prodord_updates_creations = (prodord_updates[prodord_updates['TXT30'] == 'Created'].merge(afko,how="left", left_on="OBJNR", right_on="AUFNR"))
# Add all relevant columns
timestamp_column_from_dt_tm.apply(prodord_updates, "UDATE", "UTIME", "event_timestamp")
prodord_updates['event_PRODORD'] =  prodord_updates['OBJNR']
prodord_updates['DOCTYPE_PRODORD'] = prodord_updates['event_PRODORD']
# Drop Goods Movement events, as they are extracted independently
prodord_updates = prodord_updates[(prodord_updates['TXT30'] != 'Goods movement posted') & (prodord_updates['TXT30'] != 'Delivered')]
prodord_updates['event_activity'] =  prodord_updates['TXT30'] + " Production Order"
prodord_updates['event_RESERVATION'] = None
prodord_updates['DOCTYPE_RESERVATION'] = None
# Write augmentation from above into original dataframe and drop all unrelevant columns
prodord_updates.update(prodord_updates_creations)
prodord_updates = prodord_updates[['event_timestamp','event_activity','event_PRODORD','DOCTYPE_PRODORD','event_RESERVATION','DOCTYPE_RESERVATION']]

# Process updates for Purchase Documents
object_class_mapper = {'EINKBELEG': ('PURCHORD','PO','Purchase Order','EKKO'),'BANF': ('PURCHREQ','PR','Purchase Requisition','EBAN')}
def processRow(df, index):
    if df.at[index,'OBJECTCLAS'] in object_class_mapper:
        otype, oprefix, oname, main_table = object_class_mapper.get(df.at[index,'OBJECTCLAS'])
        if main_table == df.at[index,'TABNAME'] and df.at[index,'CHNGIND'] == 'I':
            if MAKE_ADVANCED_CONNECTIONS:
                if otype == 'PURCHORD':
                    # Check if there is an purchase requisition connected with the purchase order
                    ebeln_con = ekpo[ekpo['EBELN'] == df.at[index,'OBJECTID']]
                    if len(ebeln_con) > 0:
                        _, pr_prefix, _, _ = object_class_mapper.get("BANF")
                        df.at[index,'event_PURCHREQ'] = pr_prefix + ebeln_con['BANFN'].values[0]
                        df.at[index,'DOCTYPE_PURCHREQ'] = df.at[index,'event_PURCHREQ']
            df.at[index,'event_activity'] = f"Created {oname}"
        elif df.at[index,'FNAME'] == 'FRGKE' and df.at[index,'VALUE_NEW'] == 'B':
            if MAKE_ADVANCED_CONNECTIONS:
                # Connect Reservation + Req. Material to Reject PO
                banf_list = ekpo[ekpo['EBELN'] == df.at[index,'OBJECTID']]['BANFN'].values 
                if len(banf_list) > 0:
                        if len(banf_list) > 0: print(banf_list)
                        banf = banf_list[0]
                        eban_con = eban[(eban['BANFN'] == banf) & (eban['RSNUM'] != '0000000000')]
                        if len(eban_con) > 0:
                            print('found reservation',eban_con['RSNUM'])
                            df.at[index,'event_RESERVATION'] = "RES" + eban_con['RSNUM'].values[0] + "_" + eban_con['MATNR'].values[0]
                            df.at[index,'DOCTYPE_RESERVATION'] = df.at[index,'event_RESERVATION']
            df.at[index,'event_activity'] = f" Rejected {oname}"
        elif df.at[index,'FNAME'] == 'FRGZU':
            if df.at[index,'VALUE_OLD'] == ' ' and df.at[index,'VALUE_NEW'] == 'X':
                df.at[index,'event_activity'] =  f"Released {oname} (1)"
                if MAKE_ADVANCED_CONNECTIONS:
                    if otype == 'PURCHREQ':
                        # If a reservation for this pr exists, find it and save it
                        eban_con = eban[(eban['BANFN'] == df.at[index,'OBJECTID']) & (eban['RSNUM'] != '0000000000')]
                        if len(eban_con) > 0:
                            # _, pr_prefix, _, _ = object_class_mapper.get("BANF")
                            print('found reservation',eban_con['RSNUM'])
                            df.at[index,'event_RESERVATION'] = "RES" + eban_con['RSNUM'].values[0] + "_" + eban_con['MATNR'].values[0]
                            df.at[index,'DOCTYPE_RESERVATION'] = df.at[index,'event_RESERVATION']
                            # print(ebeln_con['BANFN'].values[0])
            elif df.at[index,'VALUE_OLD'] == 'X' and df.at[index,'VALUE_NEW'] == 'XX':
                df.at[index,'event_activity'] =  f"Released {oname} (2)"

columns_to_keep = ['event_timestamp','event_activity','DOCTYPE_RESERVATION','event_RESERVATION']
changes['DOCTYPE_RESERVATION'] = None
changes['event_RESERVATION'] = None
for object_class in object_class_mapper:
    otype, oprefix, oname, main_table = object_class_mapper.get(object_class)
    changes[f"event_{otype}"] = None
    columns_to_keep.extend([f"DOCTYPE_{otype}",f"event_{otype}"])
    changes.loc[changes['OBJECTCLAS']  == object_class,f"event_{otype}"] = oprefix + changes['OBJECTID']
    changes[f"DOCTYPE_{otype}"] = changes[f"event_{otype}"]
changes.apply(lambda row: processRow(changes,row.name),axis=1)
timestamp_column_from_dt_tm.apply(changes, "UDATE", "UTIME", "event_timestamp")
changes = changes.dropna(subset=['event_activity'])[columns_to_keep]

# Process goods receipt for purchase order
timestamp_column_from_dt_tm.apply(gr_for_purord, "CPUDT", "CPUTM", "event_timestamp")
gr_for_purord['event_PURCHORD'] = "PO" + gr_for_purord['EBELN']
gr_for_purord['DOCTYPE_PURCHORD'] = gr_for_purord['event_PURCHORD']
gr_for_purord['event_activity'] = "Goods Receipt for Order"
if MAKE_ADVANCED_CONNECTIONS:
    # Connect Reservation + Req. Material to Goods Receipt for PO
    gr_for_purord['DOCTYPE_RESERVATION'] = None
    gr_for_purord['event_RESERVATION'] = None
    for index, row in gr_for_purord.iterrows():
        banf_list = ekpo[ekpo['EBELN'] == row['EBELN']]['BANFN'].values 
        if len(banf_list) > 0:
                if len(banf_list) > 0: print(banf_list)
                banf = banf_list[0]
                eban_con = eban[(eban['BANFN'] == banf) & (eban['RSNUM'] != '0000000000')]
                if len(eban_con) > 0:
                    print('found reservation',eban_con['RSNUM'])
                    gr_for_purord.at[index,'event_RESERVATION'] = "RES" + eban_con['RSNUM'].values[0] + "_" + eban_con['MATNR'].values[0]
                    gr_for_purord.at[index,'DOCTYPE_RESERVATION'] = gr_for_purord.at[index,'event_RESERVATION']
# Keep only relevant columns
gr_for_purord = gr_for_purord[['event_timestamp','event_activity','DOCTYPE_PURCHORD','event_PURCHORD','event_RESERVATION','DOCTYPE_RESERVATION'] if MAKE_ADVANCED_CONNECTIONS else ['event_timestamp','event_activity','DOCTYPE_PURCHORD','event_PURCHORD']]

# Process Goods Issue for Production Order
timestamp_column_from_dt_tm.apply(gi_for_prodord, "CPUDT_MKPF", "CPUTM_MKPF", "event_timestamp")
gi_for_prodord["event_RESERVATION"] = "RES" + gi_for_prodord['RSNUM'] + "_" + gi_for_prodord['MATNR']
# Group Goods Issue together when they are logged on same timestamp
if MAKE_ADVANCED_CONNECTIONS:
    gi_for_prodord = gi_for_prodord.groupby(['AUFNR','event_timestamp'], as_index=False)['event_RESERVATION'].agg({'event_RESERVATION':(lambda x: list(x))})
gi_for_prodord["DOCTYPE_RESERVATION"] = gi_for_prodord["event_RESERVATION"]
gi_for_prodord["event_PRODORD"] = "OR" + gi_for_prodord['AUFNR']
gi_for_prodord["DOCTYPE_PRODORD"] = gi_for_prodord["event_PRODORD"]
gi_for_prodord["event_activity"] = "Goods Issue for Production Order"
gi_for_prodord = gi_for_prodord[['event_timestamp','event_activity','event_PRODORD','DOCTYPE_PRODORD','event_RESERVATION','DOCTYPE_RESERVATION']]

# generate events df and save log
events = pd.concat([prodord_updates,changes,gr_for_purord,gi_for_prodord]).sort_values("event_timestamp")
min_extr_date = parser.parse("2022-01-21 08:15:40")
max_extr_date = parser.parse("2022-01-22 00:00:00")
events = events[(events["event_timestamp"] >= min_extr_date) & (events["event_timestamp"] <= max_extr_date)]

events.reset_index(inplace=True,drop=True)
events["event_id"] = events.index.astype(str)
ocel_exporter.apply(events, f"./auto/{datetime.now().replace(microsecond=0).isoformat()}.xmlocel")


In [ ]:
events = pd.concat([prodord_updates,changes,gr_for_purord,gi_for_prodord]).sort_values("event_timestamp")
min_extr_date = parser.parse("2022-01-21 08:15:40")
max_extr_date = parser.parse("2022-01-22 00:00:00")
print(events)
events = events[(events["event_timestamp"] >= min_extr_date) & (events["event_timestamp"] <= max_extr_date)]
print(events)
events.reset_index(inplace=True,drop=True)
events["event_id"] = events.index.astype(str)

filename = input("Filename for log")
if filename == '':
    filename = datetime.now().replace(microsecond=0).isoformat()
ocel_exporter.apply(events, f"./auto/{filename}.xmlocel")